In [4]:
import os
import time
import requests
from selenium import webdriver
import csv
import io
from PIL import Image
from webdriver_manager.chrome import ChromeDriverManager
from pathlib import Path


BASE_DIR =  Path(".").resolve()
def fetch_image_urls(query: str, max_links_to_fetch: int, wd: webdriver, sleep_between_interactions: int = 1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

        # build the google query

    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q=dog&oq=dog&gs_l=img
    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)


        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls


def persist_image(folder_path:str,url:str, counter, term=None):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert("RGB")
        file_path = os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg")
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=95)
        
        
        print(f"SUCCESS - saved {url} - as {folder_path}")
        try:
            if term is not None:
                with open(os.path.join(folder_path + " links.txt"), "a") as fl:
                    fl.write("{} retrived from {}".format(term, url))
                    fl.write("\n")
                    
        except Exception:
            print("Error writting link")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")


def search_and_download(search_term: str, target_path='./images', number_images=10):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(ChromeDriverManager().install()) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)

    counter = 0
    try:
        for elem in res:
            persist_image(target_folder, elem, counter, term=search_term)
            counter += 1
    except Exception:
        pass

# num of images you can pass it from here  by default it's 10 if you are not passing

csv_file = os.path.join(os.getcwd(), "species.csv")
with open(csv_file) as cv:
    csv_reader = list(csv.DictReader(cv))
    for row in csv_reader:
        name = row["ï»¿name"]
        print(name)
        number_images = 7
        search_and_download(search_term=name, number_images = number_images)

# number_images = 5
# for t in search_terms:
#     search_and_download(search_term=t, driver_path=DRIVER_PATH, number_images = number_images)



====== WebDriver manager ======


cats


Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Fabrice HAFASHIMANA\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
C:\Users\Fabrice HAFASHIMANA\AppData\Local\Temp\ipykernel_22400\1674341587.py:106: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(ChromeDriverManager().install()) as wd:
C:\Users\Fabrice HAFASHIMANA\AppData\Local\Temp\ipykernel_22400\1674341587.py:33: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


C:\Users\Fabrice HAFASHIMANA\AppData\Local\Temp\ipykernel_22400\1674341587.py:48: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


...

Found: 100 search results. Extracting links from 0:100
Found: 8 image links, done!
SUCCESS - saved https://media.istockphoto.com/vectors/watercolor-vector-wreath-with-green-eucalyptus-leaves-and-branches-vector-id878121918?k=20&m=878121918&s=612x612&w=0&h=qLmuqyuJxfksWPdERBU5LhDLxb8exeUYoi1G0QoRXLI= - as ./images\eucalyptus
SUCCESS - saved https://desenio.co.uk/bilder/artiklar/zoom/3393_2.jpg?imgwidth=435&qt=Eucalyptus%20branches - as ./images\eucalyptus
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRnEMqmOd7Ds_gzeP-ri67m028veIg1bzOlDQ&usqp=CAU - as ./images\eucalyptus
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTGhAnJljSrHhUnhXl6dAF19vnh-nPsgtMZIQ&usqp=CAU - as ./images\eucalyptus
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTThXsbJjDxn4TjdoOlaYOu21grtPZ3nB42ag&usqp=CAU - as ./images\eucalyptus
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSp-6SBTc6dXu628EZOSPX94CnNj1BTfXoaOQ&usqp=CAU - 



====== WebDriver manager ======


SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSbqNVC1I3aoVPHNEFcTMzlZT6eZlO8WjuLrQ&usqp=CAU - as ./images\eucalyptus
biology


Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Fabrice HAFASHIMANA\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


Found: 100 search results. Extracting links from 0:100
Found: 8 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSlHwPeREkAixW9WeOrBxS0kTwkKAxtAAnQQw&usqp=CAU - as ./images\biology
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRCoPmchJYSdCOwrHvxlNuc6JWzGHGiZW5vog&usqp=CAU - as ./images\biology
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQtWBlmsCACxZ3gQQrWXu6Nu0sf7QmkbvxZPg&usqp=CAU - as ./images\biology
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRON1tHeiqdOQGyVQP1Oa-PjB0aJhLyyaJ_IA&usqp=CAU - as ./images\biology
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRtVwlxDpLAR26l9KUomHgPtwsNadLOtHtY5w&usqp=CAU - as ./images\biology
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQxjA56t6JTXw8UrstuoeiLLqC3s_v0arB6EA&usqp=CAU - as ./images\biology
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRANqdlygHS16V3iI_qX